In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
#from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from scipy import sparse
import warnings
import time
import sys
import os
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)

In [2]:
train = pd.read_csv('jinnan_round1_train_20181227.csv', encoding = 'gb18030')
test  = pd.read_csv('jinnan_round1_testA_20181227.csv', encoding = 'gb18030')

In [3]:
train.head()

,样本id,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,收率
0,sample_1528,300,NaN,405.0,700,13:30:00,38.0,NaN,NaN,15:30:00,100,16:30:00,102,0.2,17:30:00,103.0,18:30:00,104.0,0.2,300,21:00-21:30,50.0,9.0,5.0,22:00:00,75,22:30:00,70,6:30-7:00,350.0,3.5,3.5,7:00-8:00,8:00:00,65,11:30:00,45.0,11:30-13:00,14:00-15:30,NaN,800.0,0.15,400,0.879
1,sample_1698,300,NaN,405.0,700,14:00:00,29.0,NaN,NaN,16:00:00,101,17:00:00,103,0.2,18:00:00,104.0,19:00:00,105.0,0.2,200,19:00-20:00,50.0,9.0,5.0,20:00:00,80,21:00:00,73,21:00-22:00,320.0,3.5,3.5,22:00-23:00,23:00:00,80,6:00:00,45.0,6:00-7:30,7:30-9:00,9:00-10:00,1200.0,0.15,400,0.902
2,sample_639,300,NaN,405.0,700,14:00:00,29.0,NaN,NaN,16:00:00,102,17:00:00,103,0.2,18:00:00,104.0,19:00:00,105.0,0.2,200,19:00-19:30,50.0,9.0,5.0,20:00:00,79,21:00:00,73,21:00-22:00,320.0,3.5,3.5,22:00-23:00,23:00:00,80,1:00:00,45.0,1:00-2:30,2:30-4:00,4:00-5:00,1200.0,0.15,400,0.936
3,sample_483,300,NaN,405.0,700,1:30:00,38.0,NaN,NaN,3:00:00,100,4:00:00,102,0.2,5:00:00,103.0,6:00:00,104.0,0.2,200,6:30-7:00,50.0,10.0,5.0,7:30:00,70,8:00:00,78,13:30-14:30,290.0,3.5,3.5,14:30-15:30,15:30:00,65,18:00:00,45.0,19:00-20:30,21:30-23:00,NaN,800.0,0.15,400,0.902
4,sample_617,300,NaN,405.0,700,22:00:00,29.0,NaN,NaN,0:00:00,101,1:00:00,103,0.2,2:00:00,104.0,3:00:00,105.0,0.2,200,3:00-4:00,50.0,9.0,5.0,4:00:00,80,5:00:00,73,5:00-6:00,320.0,3.5,3.5,6:00-7:00,7:00:00,80,9:00:00,45.0,9:00-10:30,10:30-12:00,12:00-13:00,1200.0,0.15,420,0.983


In [4]:
for df in [train, test]:
    df.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)

In [5]:
train.收率.value_counts().sort_index().iloc[:15]

0.624     1
0.677     1
0.803     1
0.834     2
0.846     1
0.857     1
0.866     1
0.868     5
0.870     2
0.871     1
0.874     3
0.878     1
0.879    64
0.882     1
0.884     2
Name: 收率, dtype: int64

In [6]:
good_cols = list(train.columns)
# for col in train.columns:
#     rate = train[col].value_counts(normalize=True, dropna=False).values[0]
#     if rate > 0.9:
#         good_cols.remove(col)
#         print(col,rate)

train = train[train['收率']>0.85]
        
train = train[good_cols]
good_cols.remove('收率')
test  = test[good_cols]

In [7]:
target = train['收率']
del train['收率']
data = pd.concat([train,test],axis=0,ignore_index=True)
data = data.fillna(-1)

In [8]:
def timeTranSecond(t):
    try:
        t,m,s=t.split(":")
    except:
        if t=='1900/1/9 7:00':
            return (23*3600 + 20*60)/3600
        elif t=='1900/1/1 2:30':
            return (21*3600 + 30*60)/3600
        elif t == '1900/1/21 0:00':
            return 21
        elif t == '1900/1/29 0:00':
            return 14
        elif t == '1900/1/12 0:00':
            return 12
        elif t == '1900/3/13 0:00':
            return 13
        elif t == '1900/1/22 0:00':
            return 22
        elif t == '700':
            return 7
        elif t==-1:
            return -1
        else:
            return 0
    
    try:
        tm = (int(t)*3600+int(m)*60+int(s))/3600
    except:
        return (30*60)/3600
    
    return tm
for f in ['A5','A7','A9','A11','A14','A16','A24','A26','B5','B7']:
    data[f] = data[f].apply(timeTranSecond)

def getDuration(se):
    try:
        sh,sm,eh,em=re.findall(r"\d+\.?\d*",se)
    except:
        if se == -1:
            return -1 
        
    try:
        if int(sh)>int(eh):
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600 + 24
        else:
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600
    except:
        if se=='19:-20:05':
            return 1
        elif se=='15:00-1600':
            return 1
    
    return tm

def get_start(se):
    try:
        sh, sm, eh, em = re.findall(r'\d+\.?d*', se)
    except:
        if se == -1:
            return -1
    try:
        tm = (int(eh) * 3600 + int(em) * 60)/3600
    except:
        if se == '19:-20:05':
            return 19
        elif se == '15:00-1600':
            return 15
    return tm
for f in ['A20','A28','B4','B9','B10','B11']:
    data[f+'_diff'] = data.apply(lambda df: getDuration(df[f]), axis=1)
for f in ['A20','A28','B4','B9','B10','B11']:
    data[f] = data.apply(lambda df: get_start(df[f]), axis=1)

In [9]:
time_point = ['A5','A7','A9','A11','A14','A16','A20', 'A24','A26','A28','B4','B5','B7','B9','B10','B11']
time_period = ['A20','A28','B4','B9','B10','B11']

for i, col in enumerate(time_point):
    if i == len(time_point)-2:
        break
    nex_col = time_point[i+1]
    col_name = nex_col + '-' + col
    if col not in time_period:
        data[col_name] = data[nex_col] - data[col]        
    else:
        data[col_name] = data[nex_col] - data[col] - data[col + '_diff']
    data[col_name] = data[col_name].apply(lambda x: divmod(x, 24)[1])

In [10]:
data['样本id'] = data['样本id'].apply(lambda x: int(x.split('_')[1]))

categorical_columns = [f for f in data.columns if f not in ['样本id']]
numerical_columns = [f for f in data.columns if f not in categorical_columns]

In [11]:
data.head()

,样本id,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A14,A15,A16,A17,A19,A20,A21,A22,A24,A25,A26,A27,A28,B1,B2,B4,B5,B6,B7,B8,B9,B10,B11,B12,B14,A20_diff,A28_diff,B4_diff,B9_diff,B10_diff,B11_diff,A7-A5,A9-A7,A11-A9,A14-A11,A16-A14,A20-A16,A24-A20,A26-A24,A28-A26,B4-A28,B5-B4,B7-B5,B9-B7,B10-B9
0,1528,300,-1.0,405.0,700,13.5,38.0,-1.0,-1.0,15.5,100,16.5,102.0,17.5,103.0,18.5,104.0,300,21.5,50.0,9.0,22.0,75,22.5,70.0,7.0,350.0,3.5,8.0,8.0,65,11.5,45.0,13.0,15.5,-1.0,800.0,400,0.5,0.5,1.0,1.5,1.5,-1.0,9.5,16.5,1.0,1.0,1.0,3.0,0.0,0.5,8.5,0.5,23.0,3.5,1.5,1.0
1,1698,300,-1.0,405.0,700,14.0,29.0,-1.0,-1.0,16.0,101,17.0,103.0,18.0,104.0,19.0,105.0,200,20.0,50.0,9.0,20.0,80,21.0,73.0,22.0,320.0,3.5,23.0,23.0,80,6.0,45.0,7.5,9.0,10.0,1200.0,400,1.0,1.0,1.0,1.5,1.5,1.0,9.0,17.0,1.0,1.0,1.0,1.0,23.0,1.0,1.0,0.0,23.0,7.0,1.5,0.0
2,639,300,-1.0,405.0,700,14.0,29.0,-1.0,-1.0,16.0,102,17.0,103.0,18.0,104.0,19.0,105.0,200,19.5,50.0,9.0,20.0,79,21.0,73.0,22.0,320.0,3.5,23.0,23.0,80,1.0,45.0,2.5,4.0,5.0,1200.0,400,0.5,1.0,1.0,1.5,1.5,1.0,9.0,17.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,0.0,23.0,2.0,1.5,0.0
3,483,300,-1.0,405.0,700,1.5,38.0,-1.0,-1.0,3.0,100,4.0,102.0,5.0,103.0,6.0,104.0,200,7.0,50.0,10.0,7.5,70,8.0,78.0,14.5,290.0,3.5,15.5,15.5,65,18.0,45.0,20.5,23.0,-1.0,800.0,400,0.5,1.0,1.0,1.5,1.5,-1.0,21.5,4.0,1.0,1.0,1.0,1.0,0.0,0.5,6.5,0.0,23.0,2.5,2.5,1.0
4,617,300,-1.0,405.0,700,22.0,29.0,-1.0,-1.0,0.0,101,1.0,103.0,2.0,104.0,3.0,105.0,200,4.0,50.0,9.0,4.0,80,5.0,73.0,6.0,320.0,3.5,7.0,7.0,80,9.0,45.0,10.5,12.0,13.0,1200.0,420,1.0,1.0,1.0,1.5,1.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,23.0,1.0,1.0,0.0,23.0,2.0,1.5,0.0


In [12]:
num_cols = ['A1', 'A2', 'A3', 'A4', 'A6', 'A8', 'A10', 'A12', 'A13', 'A15', 'A17', 'A18', 'A19', 'A21', 'A22', 'A23', 'A25', 'A27', 'B1', 'B2', 'B3', 'B6', 'B8', 'B12', 'B13', 'B14']

for col in num_cols:
    if col in data.columns:
        numerical_columns.append(col)
    else:
        pass

In [13]:
data['b14/a1_a3_a4_a19_b1_b12'] = data['B14']/(data['A1']+data['A3']+data['A4']+data['A19']+data['B1']+data['B12'])

numerical_columns.append('b14/a1_a3_a4_a19_b1_b12')

# del data['A1']
# del data['A3']
# del data['A4']
# categorical_columns.remove('A1')
# categorical_columns.remove('A3')
# categorical_columns.remove('A4')

In [14]:
for f in categorical_columns:
    data[f] = data[f].map(dict(zip(data[f].unique(), range(0, data[f].nunique()))))
train = data[:train.shape[0]]
test  = data[train.shape[0]:]
print(train.shape)
print(test.shape)

(1390, 59)
(150, 59)


In [15]:
train['target'] = target
train['intTarget'] = pd.cut(train['target'], 5, labels=False)
train = pd.get_dummies(train, columns=['intTarget'])
li = ['intTarget_0.0','intTarget_1.0','intTarget_2.0','intTarget_3.0','intTarget_4.0']
mean_columns = []
for f1 in categorical_columns:
    cate_rate = train[f1].value_counts(normalize=True, dropna=False).values[0]
    if cate_rate < 0.90:
        for f2 in li:
            col_name = 'B14_to_'+f1+"_"+f2+'_mean'
            mean_columns.append(col_name)
            order_label = train.groupby([f1])[f2].mean()
            train[col_name] = train['B14'].map(order_label)
            miss_rate = train[col_name].isnull().sum() * 100 / train[col_name].shape[0]
            if miss_rate > 0:
                train = train.drop([col_name], axis=1)
                mean_columns.remove(col_name)
            else:
                test[col_name] = test['B14'].map(order_label)
                
                
                
#             col_name = f1+"_"+f2+'_mean'
#             mean_columns.append(col_name)
#             order_label = train.groupby([f1])[f2].mean()
#             train[col_name] = train[f1].map(order_label)
#             miss_rate = train[col_name].isnull().sum() * 100 / train[col_name].shape[0]
#             if miss_rate > 0:
#                 train = train.drop([col_name], axis=1)
#                 mean_columns.remove(col_name)
#             else:
#                 test[col_name] = test[f1].map(order_label)
            
                        
train.drop(li+['target'], axis=1, inplace=True)
print(train.shape)
print(test.shape)

(1390, 219)
(150, 219)


In [16]:
# train_ = pd.DataFrame(train, columns=best_features)
# test_ = pd.DataFrame(test, columns=best_features)

In [17]:
# X_train = train_.values
# X_test = test_.values

In [18]:
X_train = train[mean_columns+numerical_columns].values
X_test = test[mean_columns+numerical_columns].values

In [19]:
# one hot
enc = OneHotEncoder()
for f in categorical_columns:
    enc.fit(data[f].values.reshape(-1, 1))
    X_train = sparse.hstack((X_train, enc.transform(train[f].values.reshape(-1, 1))), 'csr')
    X_test = sparse.hstack((X_test, enc.transform(test[f].values.reshape(-1, 1))), 'csr')
print(X_train.shape)
print(X_test.shape)

(1390, 2444)
(150, 2444)


In [20]:
y_train = target.values

In [21]:
# cat_params = {'learning_rate': 0.1,
#               'depth': 12,
#               'l2_leaf_reg': 10,
#               'bootstrap_type': 'Bernoulli',
#               'od_type': 'Iter',
#               'od_wait': 50,
#               'random_seed': 11,
#               'allow_writing_files': False}
# folds = KFold(n_splits=5, shuffle=True, random_state=666)
# oof_cat = np.zeros(len(train))
# predictions_cat = np.zeros(len(test))
# for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
#     print(f"fold {fold_+1}")
#     (trn_data_X, trn_data_y, val_data_X, val_data_y) = (
#         X_train.toarray()[trn_idx],
#         y_train[trn_idx],
#         X_train.toarray()[val_idx],
#         y_train[val_idx])
#     clf = CatBoostRegressor(iterations=20000, eval_metric='RMSE', use_best_model=True, **cat_params)
#     clf.fit(trn_data_X, trn_data_y,
#             eval_set=(val_data_X, val_data_y),
#             logging_level='Verbose')
#     oof_cat[val_idx] = clf.predict(val_data_X)
    
#     predictions_cat += clf.predict(X_test.toarray()) / folds.n_splits
    
# print("CV score: {:<8.8f}".format(mean_squared_error(oof_cat, target)))

In [22]:
param = {'num_leaves': 120,
         'min_data_in_leaf': 12, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8,
         "bagging_seed": 6,
         "metric": 'mse',
         "lambda_l1": 0.01,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))
feature_importance = np.zeros(X_train.shape[1])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print(f"fold {fold_+1}")
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)    
    predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
    feature_importance += clf.feature_importance() / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, target)))

fold 1
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000227199	valid_1's l2: 0.000243936
[400]	training's l2: 0.00011462	valid_1's l2: 0.000166589
[600]	training's l2: 8.1562e-05	valid_1's l2: 0.000151515
[800]	training's l2: 6.43996e-05	valid_1's l2: 0.000147521
[1000]	training's l2: 5.36803e-05	valid_1's l2: 0.000146931
[1200]	training's l2: 4.62071e-05	valid_1's l2: 0.000146595
Early stopping, best iteration is:
[1222]	training's l2: 4.55322e-05	valid_1's l2: 0.000146559
fold 2
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000224355	valid_1's l2: 0.000256856
[400]	training's l2: 0.000114961	valid_1's l2: 0.000155475
[600]	training's l2: 8.34697e-05	valid_1's l2: 0.000133645
[800]	training's l2: 6.77587e-05	valid_1's l2: 0.000125036
[1000]	training's l2: 5.7173e-05	valid_1's l2: 0.00012123
[1200]	training's l2: 4.98857e-05	valid_1's l2: 0.000119871
[1400]	training's l2: 4.43804e-05	valid_1's l2: 0.0001191

In [23]:
feature_importance.mean()

28.73641571194763

In [24]:
good_features = []
for i, imp in enumerate(feature_importance):
    if imp > 30:
        good_features.append(i)
X_train = X_train[:, good_features]
X_test = X_test[:, good_features]

In [41]:
param = {'num_leaves': 120,
         'min_data_in_leaf': 15, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.003,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8,
         "bagging_seed": 6,
         "metric": 'mse',
         "lambda_l1": 0.05,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print(f"fold {fold_+1}")
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, target)))

fold 1
Training until validation scores don't improve for 200 rounds.
[200]	training's l2: 0.000389059	valid_1's l2: 0.000361796
[400]	training's l2: 0.000227889	valid_1's l2: 0.000238018
[600]	training's l2: 0.000164657	valid_1's l2: 0.00019131
[800]	training's l2: 0.000135854	valid_1's l2: 0.000171933
[1000]	training's l2: 0.000116981	valid_1's l2: 0.000160923
[1200]	training's l2: 0.000103647	valid_1's l2: 0.000154169
[1400]	training's l2: 9.42069e-05	valid_1's l2: 0.000149725
[1600]	training's l2: 8.72658e-05	valid_1's l2: 0.000146528
[1800]	training's l2: 8.20169e-05	valid_1's l2: 0.000144659
[2000]	training's l2: 7.79439e-05	valid_1's l2: 0.000143332
[2200]	training's l2: 7.47274e-05	valid_1's l2: 0.000142486
[2400]	training's l2: 7.18296e-05	valid_1's l2: 0.000141829
[2600]	training's l2: 6.93141e-05	valid_1's l2: 0.000141407
[2800]	training's l2: 6.71248e-05	valid_1's l2: 0.000141097
[3000]	training's l2: 6.52139e-05	valid_1's l2: 0.000140954
[3200]	training's l2: 6.34399e-05	v

[1600]	training's l2: 8.82801e-05	valid_1's l2: 0.000157543
[1800]	training's l2: 8.36151e-05	valid_1's l2: 0.00015501
[2000]	training's l2: 7.9821e-05	valid_1's l2: 0.000153001
[2200]	training's l2: 7.66624e-05	valid_1's l2: 0.000151252
[2400]	training's l2: 7.40481e-05	valid_1's l2: 0.000150044
[2600]	training's l2: 7.18322e-05	valid_1's l2: 0.000148818
[2800]	training's l2: 6.984e-05	valid_1's l2: 0.000147989
[3000]	training's l2: 6.80104e-05	valid_1's l2: 0.000147408
[3200]	training's l2: 6.63706e-05	valid_1's l2: 0.000146763
[3400]	training's l2: 6.48648e-05	valid_1's l2: 0.000146223
[3600]	training's l2: 6.35131e-05	valid_1's l2: 0.000145736
[3800]	training's l2: 6.22624e-05	valid_1's l2: 0.000145147
[4000]	training's l2: 6.11032e-05	valid_1's l2: 0.000144697
[4200]	training's l2: 6.0038e-05	valid_1's l2: 0.000144411
[4400]	training's l2: 5.90301e-05	valid_1's l2: 0.000144185
[4600]	training's l2: 5.80957e-05	valid_1's l2: 0.000143992
[4800]	training's l2: 5.72132e-05	valid_1's l

In [26]:
xgb_params = {'eta': 0.005, 'max_depth': 12, 'subsample': 0.8, 'colsample_bytree': 0.8, 
          'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}

folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, target)))

fold n°1
[0]	train-rmse:0.422529	valid_data-rmse:0.42369
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.256766	valid_data-rmse:0.257975
[200]	train-rmse:0.156495	valid_data-rmse:0.157566
[300]	train-rmse:0.095866	valid_data-rmse:0.097085
[400]	train-rmse:0.0593	valid_data-rmse:0.060766
[500]	train-rmse:0.0373	valid_data-rmse:0.039019
[600]	train-rmse:0.024096	valid_data-rmse:0.026275
[700]	train-rmse:0.016155	valid_data-rmse:0.019028
[800]	train-rmse:0.011387	valid_data-rmse:0.015174
[900]	train-rmse:0.008493	valid_data-rmse:0.013241
[1000]	train-rmse:0.006689	valid_data-rmse:0.012338
[1100]	train-rmse:0.005496	valid_data-rmse:0.011944
[1200]	train-rmse:0.0047	valid_data-rmse:0.011802
[1300]	train-rmse:0.004116	valid_data-rmse:0.011766
[1400]	train-rmse:0.003665	valid_data-rmse:0.01178
[1500]	train-rmse:0.003269	valid_data-rmse:0.011823
Stopping. Best iteratio

In [42]:
# oof_h2o = pd.read_csv('oof_h2o.csv', header=None).values.reshape(-1)
# predictions_h2o = pd.read_csv('predictions_h2o.csv', header=None).values.reshape(-1)
# print("CV score: {:<8.8f}".format(mean_squared_error(oof_h2o, target)))

In [43]:
train_stack = np.vstack([oof_lgb, oof_xgb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb]).transpose()

In [44]:
pd.DataFrame(train_stack).corr()

,0,1
0,1.000000,0.985151
1,0.985151,1.000000


In [45]:
folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4590)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,target)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values
    
    clf_3 = BayesianRidge()
    clf_3.fit(trn_data, trn_y)
    print(clf_3.coef_)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
mean_squared_error(target.values, oof_stack)

fold 0
[0.44186566 0.57170683]
fold 1
[0.38277962 0.61666664]
fold 2
[0.45654127 0.54837778]
fold 3
[0.50591204 0.49918328]
fold 4
[0.45185683 0.5581124 ]
fold 5
[0.50050107 0.51416893]
fold 6
[0.42206392 0.58194952]
fold 7
[0.51253751 0.49555955]
fold 8
[0.38429712 0.61819121]
fold 9
[0.42419191 0.57980485]


0.0001232886103972429

In [46]:
sub_df = pd.read_csv('jinnan_round1_submit_20181227.csv', header=None)
sub_df[1] = predictions
sub_df[1] = sub_df[1].apply(lambda x:round(x, 3))
sub_df.to_csv("submit.csv", index=False, header=None)

In [47]:
(sub_df[1]**2).sum()/len(sub_df.index)/2

0.4278535233333333

In [48]:
sub_df

,0,1
0,sample_1656,0.907
1,sample_1548,0.879
2,sample_769,0.933
3,sample_1881,0.903
4,sample_1807,0.927
5,sample_145,0.927
6,sample_1212,0.935
7,sample_944,0.896
8,sample_829,0.934
9,sample_616,0.925


In [ ]:
# def modeling_cross_validation(params, X, y, nr_folds=5):
    
#     oof_preds = np.zeros(X.shape[0])
#     # Split data with kfold
#     folds = KFold(n_splits=nr_folds, shuffle=False, random_state=4096)
    
#     for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
#         print("fold n°{}".format(fold_+1))
#         trn_data = lgb.Dataset(X[trn_idx], y[trn_idx])
#         val_data = lgb.Dataset(X[val_idx], y[val_idx])

#         num_round = 20000
#         clf = lgb.train(params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 100)
#         oof_preds[val_idx] = clf.predict(X[val_idx], num_iteration=clf.best_iteration)

#     score = mean_squared_error(oof_preds, target)
    
#     return  score/2

In [ ]:
# def featureSelect(init_cols):
#     params = {'num_leaves': 120,
#              'min_data_in_leaf': 30, 
#              'objective':'regression',
#              'max_depth': -1,
#              'learning_rate': 0.05,
#              "min_child_samples": 30,
#              "boosting": "gbdt",
#              "feature_fraction": 0.9,
#              "bagging_freq": 1,
#              "bagging_fraction": 0.9 ,
#              "bagging_seed": 11,
#              "metric": 'mse',
#              "lambda_l1": 0.02,
#              "verbosity": -1}
#     best_cols = init_cols.copy()
#     best_score = modeling_cross_validation(params, train[init_cols].values, target.values, nr_folds=5)
#     print("初始CV score: {:<8.8f}".format(best_score))
#     for f in init_cols:

#         best_cols.remove(f)
#         score = modeling_cross_validation(params, train[best_cols].values, target.values, nr_folds=5)
#         diff = best_score - score
#         print('-'*10)
#         if diff > 0.0000002:
#             print("当前移除特征: {}, CV score: {:<8.8f}, 最佳cv score: {:<8.8f}, 有效果,删除！！".format(f,score,best_score))
#             best_score = score
#         else:
#             print("当前移除特征: {}, CV score: {:<8.8f}, 最佳cv score: {:<8.8f}, 没效果,保留！！".format(f,score,best_score))
#             best_cols.append(f)
#     print('-'*10)
#     print("优化后CV score: {:<8.8f}".format(best_score))
    
#     return best_cols
    
# best_features = featureSelect(train.columns.tolist())